In [40]:
import tensorflow as tf
import scipy.io as sio
import numpy as np
import time
from datetime import datetime
import os
import h5py as h5
import utils
import tf_utils
import parser_ops
import masks.ssdu_masks as ssdu_masks
import UnrollNet

In [47]:
file_name = '/srv/share4/ksarangmath3/mri/data/singlecoil_train/file1000017.h5'


In [48]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [49]:
f = h5.File(file_name, "r")

In [52]:
kspace_train = f['kspace'][:]
# kspace_train = np.expand_dims(kspace_train, axis=1)
kspace_shape = kspace_train.shape

In [54]:
np.expand_dims(kspace_train,axis=3).shape

(34, 640, 372, 1)

In [29]:
rec_rss = f['reconstruction_rss'][:]
rec_rss.shape

(34, 320, 320)

In [30]:
# Ground truth
rec_esc = f['reconstruction_esc'][:]
rec_esc.shape

(34, 320, 320)

In [31]:
from fastmri.data.subsample import create_mask_for_mask_type

In [32]:
mask_fn = create_mask_for_mask_type("random",[0.08], [4])

In [33]:
mask = mask_fn(kspace_shape, 1)

In [34]:
mask.shape

torch.Size([1, 1, 640, 1])

In [35]:
kspace_shape

(34, 1, 640, 372)

In [36]:
sens_maps = bart.bart(1, f"ecalib -d0 -m1 -r", kspace_train)

NameError: name 'bart' is not defined

In [1]:
# import tensorflow as tf
import scipy.io as sio
import numpy as np
import time
from datetime import datetime
import os
import h5py as h5
import utils
import tf_utils
import parser_ops
import masks.ssdu_masks as ssdu_masks
import UnrollNet
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.INFO)

from fastmri.data.subsample import create_mask_for_mask_type


# if __name__ == "main":
parser = parser_ops.get_parser()
args = parser.parse_args()
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

save_dir ='saved_models'
directory = os.path.join(save_dir, 'SSDU_' + args.data_opt + '_' +str(args.epochs)+'Epochs_Rate'+ str(args.acc_rate) +\
                         '_' + str(args.nb_unroll_blocks) + 'Unrolls_' + args.mask_type+'Selection' )

if not os.path.exists(directory):
    os.makedirs(directory)

# tf.logging.info('\n create a test model for the testing')
# test_graph_generator = tf_utils.test_graph(directory)

#...........................................................................d....
start_time = time.time()
tf.logging.info('.................SSDU Training.....................')
ops.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

# .......................Load the Data..........................................
tf.logging.info('\n Loading {args.data_opt} data, acc rate : {args.acc_rate} mask type : {args.mask_type}')

##TODO(): Use all the training data, not just one file
kspace_dir = '/srv/share4/ksarangmath3/mri/data/singlecoil_train/file1000017.h5'
kspace_train = h5.File(kspace_dir, "r")['kspace'][:]

## Adding dimension for coil
kspace_train = np.expand_dims(kspace_train,3)
kspace_shape = kspace_train.shape

##TODO(): GET actual sensitivity maps 
sens_maps = np.ones(kspace_shape)

tf.logging.info('\n Normalize the kspace to 0-1 region')
for ii in range(np.shape(kspace_train)[0]):
    kspace_train[ii, :, :, :] = kspace_train[ii, :, :, :] / np.max(np.abs(kspace_train[ii, :, :, :][:]))

nSlices, *_ = kspace_train.shape

#TODO(): Use generated masks
# mask_fn = create_mask_for_mask_type("random",[0.08], [4])
# original_mask = mask_fn((1,34,640,372), 1)
original_mask = np.zeros((args.nrow_GLOB, args.ncol_GLOB))
original_mask[:,:10] = 1

tf.logging.info(f'\n size of kspace: {kspace_train.shape}, maps: {sens_maps.shape}, mask: {original_mask.shape}')


trn_mask, loss_mask = np.empty((nSlices, args.nrow_GLOB, args.ncol_GLOB), dtype=np.complex64), \
                      np.empty((nSlices, args.nrow_GLOB, args.ncol_GLOB), dtype=np.complex64)

nw_input = np.empty((nSlices, args.nrow_GLOB, args.ncol_GLOB), dtype=np.complex64)
ref_kspace = np.empty((nSlices, args.nrow_GLOB, args.ncol_GLOB, args.ncoil_GLOB), dtype=np.complex64)

tf.logging.info('\n create training and loss masks and generate network inputs... ')
ssdu_masker = ssdu_masks.ssdu_masks()
for ii in range(nSlices):
    if np.mod(ii, 50) == 0:
        tf.logging.info(f'\n Iteration: {ii}')

    if args.mask_type == 'Gaussian':
        # tf.logging.info(original_mask)
        trn_mask[ii, ...], loss_mask[ii, ...] = ssdu_masker.Gaussian_selection(kspace_train[ii], original_mask, num_iter=ii)

    elif args.mask_type == 'Uniform':
        trn_mask[ii, ...], loss_mask[ii, ...] = ssdu_masker.uniform_selection(kspace_train[ii], original_mask, num_iter=ii)
        
    else:
        raise ValueError('Invalid mask selection')

    sub_kspace = kspace_train[ii] * np.tile(trn_mask[ii][..., np.newaxis], (1, 1, args.ncoil_GLOB))
    ref_kspace[ii, ...] = kspace_train[ii] * np.tile(loss_mask[ii][..., np.newaxis], (1, 1, args.ncoil_GLOB))
    nw_input[ii, ...] = utils.sense1(sub_kspace, sens_maps[ii, ...])

print(sub_kspace)

usage: ipykernel_launcher.py [-h] [--acc_rate ACC_RATE] [--epochs EPOCHS]
                             [--learning_rate LEARNING_RATE]
                             [--batchSize BATCHSIZE]
                             [--nb_unroll_blocks NB_UNROLL_BLOCKS]
                             [--nb_res_blocks NB_RES_BLOCKS]
                             [--CG_Iter CG_ITER] [--data_opt DATA_OPT]
                             [--nrow_GLOB NROW_GLOB] [--ncol_GLOB NCOL_GLOB]
                             [--ncoil_GLOB NCOIL_GLOB]
                             [--mask_type {Gaussian,Uniform}] [--rho RHO]
ipykernel_launcher.py: error: unrecognized arguments: -f /nethome/bdevnani3/.local/share/jupyter/runtime/kernel-9f597231-a92f-41fa-8220-9b913bda16b2.json


SystemExit: 2